# Using CensusReporter to Make Geographic Selections 
## By: Nicole Miller 
### This project illustrates how APIs can be used to make geographic target selections using data from the Census

In [1]:
#importing the packages necessary to complete our analysis 
import pandas as pd
import json  
import requests 
import time 


In [2]:
#this assigns the filename we're trying to load in to a string variable
msas = 'msas.csv'

In [3]:
#this uses the pandas package to load the csv file into a pandas dataframe
df = pd.read_csv(msas)  

In [4]:
#table ids I want to download 
tableids = ['B01001', 'B03002', 'B19001'] 
#table that represent age, race, income respectively 

In [5]:
#joining list together for table id
tableidstring = ','.join(tableids) 

In [6]:
#setting cbsa as the index
msas = df.set_index("CBSA", drop= False)

In [7]:
#setting empty variables 
urllist = []  
geoid= 0 
geoidlist = []

In [8]:
#iterating through each index
for index, row in msas.iterrows(): 
    geoid = '31000US' + str(index)
    requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidstring, geoid) 
    urllist.append(requesturl) 
    #appending a geoid to a list of geoids
    geoidlist.append(str(geoid))

In [9]:
#adding a URL column     
msas['URL'] = urllist 

In [10]:
#making new dataframe
alldf = pd.DataFrame()
alldf['URL'] = msas['URL'] 

In [11]:
#appending the geoids to the dataframes
alldf['GEO_ID'] = geoidlist 
msas['GEO_ID'] = geoidlist

In [12]:
#getting empty lists for the age, income, and race for every geoid  
#Since URLlist is in the same order, it will be simple to transform 
age = [] 
income = [] 
race = []  

In [13]:
#There are lots of code for the ages we are interested in, so we made a list that we can itterate through 
age_get = ['B01001007','B01001008','B01001009', 'B01001010','B01001011','B01001012']   

In [14]:
#Iterating through each URL list 
for element in urllist:  
    #Use the current URL to get the json
    loadedjson = requests.get(element) 
    loadedjson.json()
    parsedjson= json.loads(loadedjson.text) 
    #This is the CBSA of the current URL, just taken off the end 
    locstr = element[-12:] 
    
    #Creating three dataframes of the data we need using the CBSA gleaned from table
    agedf= pd.DataFrame.from_dict(parsedjson['data'][locstr]['B01001'])
    racedf= pd.DataFrame.from_dict(parsedjson['data'][locstr]['B03002'])
    incomedf= pd.DataFrame.from_dict(parsedjson['data'][locstr]['B19001']) 

    #Iterating over that list of age codes and add it to age_sum
    age_sum = 0 
    for amt in age_get: 
        age_sum = agedf['estimate'][amt] + age_sum 
    
    #Appending the sum to the age list 
    age.append(age_sum)
    #Appending the income data to the income list 
    income.append(incomedf['estimate']['B19001011']+ incomedf['estimate']['B19001012']) 
    #Appending the race data to the race list (Hispanics only)
    race.append(racedf['estimate']['B03002012'])
    #Slows down the for loop so the website doesn't crash  
        #This for loop will take time so pull data without the website bugging out 
    time.sleep(-time.time()%1) 

In [15]:
#Appending the different lists to columns in out dataframe 
alldf['Age'] = age 
alldf['Income'] = income 
alldf['Race']= race 

In [16]:
#pulling in the Name of geography region 
alldf['Name'] = msas['NAME']

In [17]:
#What city has the highest count of the audience you chose for parameter #1? (income)
print (alldf.loc[alldf['Income'].idxmax()])
#New York-Newark-Jersey City, NY-NJ-PA has the highest count for the income we are interested in 

URL       http://api.censusreporter.org/1.0/data/show/la...
GEO_ID                                         31000US35620
Age                                             2.40159e+06
Income                                          1.03819e+06
Race                                             4.9944e+06
Name                  New York-Newark-Jersey City, NY-NJ-PA
Name: 35620, dtype: object


In [18]:
#What city has the highest count of the audience you chose for parameter #2? (age)
print (alldf.loc[alldf['Age'].idxmax()])
##New York-Newark-Jersey City, NY-NJ-PA has the highest count for the age range we are interested in

URL       http://api.censusreporter.org/1.0/data/show/la...
GEO_ID                                         31000US35620
Age                                             2.40159e+06
Income                                          1.03819e+06
Race                                             4.9944e+06
Name                  New York-Newark-Jersey City, NY-NJ-PA
Name: 35620, dtype: object


In [19]:
#What city has the highest count of the audience you chose for parameter #3? (race)
print (alldf.loc[alldf['Race'].idxmax()])
#Los Angeles-Long Beach-Anaheim, CA has the highest count for the race we are interested in 

URL       http://api.censusreporter.org/1.0/data/show/la...
GEO_ID                                         31000US31080
Age                                             1.72147e+06
Income                                               708153
Race                                            6.03149e+06
Name                     Los Angeles-Long Beach-Anaheim, CA
Name: 31080, dtype: object


In [20]:
#Highest average mean for age 
#Sum of all Ages
TotalAge = alldf['Age'].sum()

#percentage 
alldf['Age Percentage'] = alldf['Age']/TotalAge * 100

#max mean percentage 
print (alldf.loc[alldf['Age Percentage'].idxmax()]) 

URL               http://api.censusreporter.org/1.0/data/show/la...
GEO_ID                                                 31000US35620
Age                                                     2.40159e+06
Income                                                  1.03819e+06
Race                                                     4.9944e+06
Name                          New York-Newark-Jersey City, NY-NJ-PA
Age Percentage                                              7.15063
Name: 35620, dtype: object


In [21]:
#Highest average mean for income
#Sum of all Ages
TotalIncome = alldf['Income'].sum()

#percentage 
alldf['Income Percentage'] = alldf['Income']/TotalIncome * 100

#max mean percentage 
print (alldf.loc[alldf['Income Percentage'].idxmax()]) 

URL                  http://api.censusreporter.org/1.0/data/show/la...
GEO_ID                                                    31000US35620
Age                                                        2.40159e+06
Income                                                     1.03819e+06
Race                                                        4.9944e+06
Name                             New York-Newark-Jersey City, NY-NJ-PA
Age Percentage                                                 7.15063
Income Percentage                                              5.82497
Name: 35620, dtype: object


In [22]:
#Highest average mean for race
#Sum of all Ages
TotalRace = alldf['Race'].sum()

#percentage 
alldf['Race Percentage'] = alldf['Race']/TotalRace * 100

#max mean percentage 
print (alldf.loc[alldf['Race Percentage'].idxmax()]) 

URL                  http://api.censusreporter.org/1.0/data/show/la...
GEO_ID                                                    31000US31080
Age                                                        1.72147e+06
Income                                                          708153
Race                                                       6.03149e+06
Name                                Los Angeles-Long Beach-Anaheim, CA
Age Percentage                                                  5.1256
Income Percentage                                              3.97324
Race Percentage                                                11.2988
Name: 31080, dtype: object


In [23]:
#Highest Overall Percentage across all categories  
alldf['Overall Percentage'] = alldf['Age Percentage']+alldf['Income Percentage']+alldf['Race Percentage']
print (alldf.loc[alldf['Overall Percentage'].idxmax()]) 

URL                   http://api.censusreporter.org/1.0/data/show/la...
GEO_ID                                                     31000US35620
Age                                                         2.40159e+06
Income                                                      1.03819e+06
Race                                                         4.9944e+06
Name                              New York-Newark-Jersey City, NY-NJ-PA
Age Percentage                                                  7.15063
Income Percentage                                               5.82497
Race Percentage                                                 9.35602
Overall Percentage                                              22.3316
Name: 35620, dtype: object
